# Recommendations Systems
## Assignment 3:  Neural Collaborative Filtering

**By:**  
Group 16

<br><br>

**The goal of this assignment is to:**
- Understand the concept of recommendations based on implicit data which is very common in real life.
- Understand how DL components can be used to implement a collaborative filtering & hybrid approach recommenders.
- Understand pros&cons comparing to other recommender system approaches.
- Practice recommender system training and evaluation.

**Instructions:**
- Students will form teams of two people each, and submit a single homework for each team.
- The same score for the homework will be given to each member of the team.
- Your solution in the form of an Jupyter notebook file (with extension ipynb).
- Images/Graphs/Tables should be submitted inside the notebook.
- The notebook should be runnable and properly documented. 
- Please answer all the questions and include all your code.
- English only.

**Submission:**
- Submission of the homework will be done via Moodle by uploading a link to google colab.
- The homwork needs to be entirely in English.
- The deadline for submission is on Moodle.

**Requirements:**  
- Python 3.6+ should be used. 
- You may use Torch/Keras/TF packeges.
- You should implement the recommender system by yourself using only basic Python libraries (such as numpy).

**LINKS:**
- <a href='https://github.com/hexiangnan/neural_collaborative_filtering/tree/master/Data'>Dataset</a>
- <a href='https://github.com/hexiangnan/neural_collaborative_filtering'>Repository</a>
- <a href='https://towardsdatascience.com/paper-review-neural-collaborative-filtering-explanation-implementation-ea3e031b7f96'>Blog Post Review</a>
<br>

**Google <a href='https://colab.research.google.com/'>Colaboratory</a>**  
        
    This is a great academic tool for students. Instead of installing and running "everything" on your Laptop - which probably will take you a lot of time - you can use Google Colab.  
    Basically, you can use it for all your Python needs.  

**PyTorch <a href='https://pytorch.org/tutorials/beginner/basics/intro.html'>Tutorials</a>**   
    
    Just follow steps 0-7 and you will have the basics skills to understand, build, and run DL recommender models. 

**Keras Kaggle's <a href='https://www.kaggle.com/learn/intro-to-deep-learning'>intro-to-deep-learning</a>**  
    
    This will give you a quick idea of what DL is, and how to utilize it.  
    They're using TensorFlow, while in our MLDL program we're using PyTorch.  




**Grading:**

- Q1 - 20 points - Dataset Preparation
- Q2 - 50 points - Neural Collaborative Filtering
- Q3 - 30 points - Loss Function

`Total: 100`

<br><br><br>

**Prerequisites**

In [30]:
%pip install torch torchvision --quiet

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


**Imports**

In [31]:
# basic
import os 
import sys
import math
import heapq
import argparse
from time import time
import multiprocessing

# general
import warnings
import numpy as np
import scipy as sp
import pandas as pd
import scipy.sparse as sp

# visual
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

# visual 3D
from mpl_toolkits import mplot3d

# notebook
from IPython.display import display, HTML


# torch
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Sequential
from torch.nn import Sigmoid,ReLU
from torch.nn import Embedding,Linear,Dropout
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor,Compose
from torch.optim import SparseAdam,Adam,Adagrad,SGD

# colab
# from google.colab import drive  

**Hide Warnings**

In [32]:
warnings.filterwarnings('ignore')

**Disable Autoscrolling**

In [33]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};

<IPython.core.display.Javascript object>

<br><br><br>
<br><br><br>
<br><br><br>

## Question 1: Dataset Preparation (Ingestion)

---


<br><br>

This implementation contains one file for training and two files for testing:   
- ml-1m.train.rating   
- ml-1m.test.rating  
- ml-1m.test.negative   

<br>
(feel free to use visual explanations)
<br>

a. **Explain** the role and structure of each file and how it was created from the original <a href='https://github.com/hexiangnan/neural_collaborative_filtering/tree/master/Data'>MovieLens 1M rating dataset</a>.

- train rating file consists of positive instances. Each line in the file is of the following format: user_id item_id rating time_stamp .
- test rating file consists of positive instances (basically last sample for each user of interaction which isn't in train). Each line in the file is of the following format: user_id item_id rating time_stamp . One line per user.
- Negative sample file is file which each line corresponds to user id and test item(positive) and 99 items ids which were negatively sampled, format: (user_id,item_id) , negative sample1,negative sample2, ...

b. **Explain** how the training dataset is created.

- We iterate through each line of the train file.
- We find the number of items and users.
- We create a sparse matrix with NXM (users X items)
- We go through the file again and for each user item pair in the file line we check if there has been rating with a value greater than 0.
- If there has been a positive rating we add a binary indicator of implicit feedback (1).

c. **Explain** how the test dataset is created.

- test ratings list:
 is a list with tuples of user, a positively rated item(which isn't in the train set).
- negative sampling list:
Each line in negative file contains 99 negative samples(the id of the user matches the index of the line of test ratings file).

Then during the evaluation we will evaluate the 99 negative samples and the positive sample from the test list.

#### Data Preperations:

In [34]:
# urls
train_url = "https://github.com/hexiangnan/neural_collaborative_filtering/blob/master/Data/ml-1m.train.rating?raw=true"
test_url = "https://github.com/hexiangnan/neural_collaborative_filtering/blob/master/Data/ml-1m.test.rating?raw=true"
test_neg_url = "https://github.com/hexiangnan/neural_collaborative_filtering/blob/master/Data/ml-1m.test.negative?raw=true"


In [64]:
import requests

def get_num_users_and_items(data_lines):
    num_users = 0
    num_items = 0
    for line in data_lines:
        line_values = line.split("\t")
        user_id = int(line_values[0])
        item_id = int(line_values[1])
        num_users = max(num_users, user_id)
        num_items = max(num_items, item_id)

    return num_users, num_items

def generate_matrix(data_lines, num_users, num_items):
    matrix = sp.dok_matrix((num_users + 1, num_items + 1), dtype=np.float32)
    for line in data_lines:
        line_values = line.split("\t")
        user_id = int(line_values[0])
        item_id = int(line_values[1])
        rating = float(line_values[2])
        if rating > 0:
            matrix[user_id, item_id] = 1.0
    
    return matrix
    

def load_data_as_matrix(url):
    # download data with requests
    response = requests.get(url)
    # Read as a text file
    data = response.text
    # Split by lines
    data_lines = data.splitlines()
    # Get number of users and items
    num_users, num_items = get_num_users_and_items(data_lines)\
    # Construct matrix
    mat = generate_matrix(data_lines, num_users, num_items)
    
    return mat

def load_negatives_vector(url):
    # download data with requests
    response = requests.get(url)
    # Read as a text file
    data = response.text
    # Split by lines
    data_lines = data.splitlines()
    # Construct vector
    vector = []
    for line in data_lines:
        line_values = line.split("\t")
        vector.append([int(x) for x in line_values[1:]])
    
    return vector

def load_data_as_list(url):
    res = []
    # download data with requests
    response = requests.get(url)
    # Read as a text file
    data = response.text
    # Split by lines
    data_lines = data.splitlines()
    
    for line in data_lines:
        line_values = line.split("\t")
        user_id = int(line_values[0])
        item_id = int(line_values[1])
        res.append([user_id, item_id])

    return res
     


In [62]:
train_matrix = load_data_as_matrix(train_url)
test_ratings = load_data_as_list(test_url)
test_neg_vector = load_negatives_vector(test_neg_url)

num_users, num_items = train_matrix.shape

In [63]:
# print shapes of train and test matrices
print("Train matrix shape: ", train_matrix.shape)
print("Test matrix shape: ", len(test_ratings))

# print shape of test negative vector
print("Test negative vector shape: ", len(test_neg_vector))
print(len(test_neg_vector[0]))

Train matrix shape:  (6040, 3706)
Test matrix shape:  6040
Test negative vector shape:  6040
99


<br><br><br>
<br><br><br>
<br><br><br>

## Question 2: Neural Collaborative Filtering 
<br><br>

## a. Build the following four models using the neural collaborative filtering approach: 
- Matrix Factorization (MF)
- Multi layer perceptron (MLP)
- Generalized Matrix Factorization (GMF) 
- NeuroMatrixFactorization (NMF)

**For each model, use the best hyper-parameters suggested in the neuMF paper.**

<br><br><br><br>
#### Matrix Factorization (MF)  
<br>

In [37]:
class MF(nn.Module):
    def __init__(self, num_users, num_items, embedding_size=32):
        super(MF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.emb_size = embedding_size
        self.emb_item = nn.Embedding(num_embeddings=num_items, embedding_dim=self.emb_size)
        self.emb_user = nn.Embedding(num_embeddings=num_users, embedding_dim=self.emb_size)
        self._init_weights()

    def _init_weights(self):
        nn.init.normal_(self.emb_user.weight, std=0.01)
        nn.init.normal_(self.emb_item.weight, std=0.01)

    def forward(self, users, items):
        emb_user = self.emb_user(users)
        emb_item = self.emb_item(items)
        return emb_user @ emb_item.T

In [38]:
model_MF = MF(num_users, num_items, embedding_size=32)

Model's architecture:

In [39]:
# display/print the model architecture
print(model_MF)

MF(
  (emb_item): Embedding(3706, 32)
  (emb_user): Embedding(6040, 32)
)


<br><br><br><br><br><br>
#### Multi Layer Perceptron (MLP)

In [40]:
class MLP(nn.Module):
    def __init__(self, num_users, num_items, embedding_size=16, mlp_layers_sizes=[32, 16, 8], dropout=0.1):
        super(MLP, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_size = embedding_size
        self.mlp_layers_sizes = mlp_layers_sizes
        self.dropout = dropout

        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=embedding_size)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=embedding_size)
        self.mlp_layers = nn.Sequential(
            nn.Linear(2 * embedding_size, mlp_layers_sizes[0]),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(mlp_layers_sizes[0], mlp_layers_sizes[1]),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(mlp_layers_sizes[1], mlp_layers_sizes[2]),
        )
        self.last_hidden = nn.Sequential(
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(mlp_layers_sizes[2], 1)
        )
        self.activation = nn.Sigmoid()

    def forward(self, user_ids, item_ids):
        user_embedding = self.user_embedding(user_ids)
        item_embedding = self.item_embedding(item_ids)
        input_vector = torch.cat([user_embedding, item_embedding], dim=1)
        output = self.mlp_layers(input_vector)
        output = self.activation(output)
        return output

        

In [41]:
model_MLP = MLP(num_users, num_items, embedding_size=16, mlp_layers_sizes=[32, 16, 8], dropout=0.1)

Model's architecture:

In [42]:
# display/print the model architecture
print(model_MLP)

MLP(
  (user_embedding): Embedding(6040, 16)
  (item_embedding): Embedding(3706, 16)
  (mlp_layers): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=16, out_features=8, bias=True)
  )
  (last_hidden): Sequential(
    (0): ReLU()
    (1): Dropout(p=0.1, inplace=False)
    (2): Linear(in_features=8, out_features=1, bias=True)
  )
  (activation): Sigmoid()
)


<br><br><br><br><br><br>
####Generalized Matrix Factorization (GMF)

In [43]:
class GMF(nn.Module):
    def __init__(self, num_users, num_items, embedding_size=32):
        super(GMF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.emb_size = embedding_size
        self.emb_item = nn.Embedding(num_embeddings=num_items, embedding_dim=self.emb_size)
        self.emb_user = nn.Embedding(num_embeddings=num_users, embedding_dim=self.emb_size)
        self.hidden = torch.nn.Linear(self.emb_size, 1) 
        self.activation = nn.Sigmoid()
        self._init_weights()

    def _init_weights(self):
        nn.init.normal_(self.emb_user.weight, std=0.01)
        nn.init.normal_(self.emb_item.weight, std=0.01)

    def forward(self, users, items):
        emb_user = self.emb_user(users)
        emb_item = self.emb_item(items)
        element_wise = emb_user * emb_item
        output = self.hidden(element_wise)
        output = self.activation(output)
        


In [44]:
model_GMF = GMF(num_users, num_items,embedding_size = 32)

Model's architecture:

In [45]:
# display/print the model architecture
print(model_GMF)

GMF(
  (emb_item): Embedding(3706, 32)
  (emb_user): Embedding(6040, 32)
  (hidden): Linear(in_features=32, out_features=1, bias=True)
  (activation): Sigmoid()
)


<br><br><br><br><br><br>
#### NeuroMatrixFactorization (NMF)


In [48]:
# Note for the simplicity of the implementation I've decided not to take mlp and gmf as backbones(like it was done in the paper)
class NCF(nn.Module):
    def __init__(
        self,
        num_users,
        num_items,
        embedding_size=32,
        mlp_embedding_size=16,
        mlp_layers_sizes=[32, 16, 8],
        dropout=0.1,
    ):
        super(NCF, self).__init__()
        self.mlp = MLP(
            num_users,
            num_items,
            embedding_size=mlp_embedding_size,
            mlp_layers_sizes=mlp_layers_sizes,
            dropout=dropout,
        )
        self.gmf = GMF(num_users, num_items, embedding_size=embedding_size)
        self.neu_mf = nn.Linear(mlp_layers_sizes[-1] + embedding_size, 1)
        self.activation = nn.Sigmoid()
        self.mute_last_layers()

    def mute_last_layers(self):
        self.mlp.activation = nn.Identity()
        self.gmf.activation = nn.Identity()
        self.gmf.hidden = nn.Identity()
        self.mlp.last_hidden = nn.Identity()

    def forward(self, users, items):
        mlp_output = self.mlp(users, items)
        gmf_output = self.gmf(users, items)
        output = torch.cat([mlp_output, gmf_output], dim=1)
        output = self.neu_mf(output)
        output = self.activation(output)
        return output


In [49]:
model_NMF = NCF(num_users, num_items)

Model's architecture:

In [50]:
# display/print the model architecture
print(model_NMF)

NCF(
  (mlp): MLP(
    (user_embedding): Embedding(6040, 16)
    (item_embedding): Embedding(3706, 16)
    (mlp_layers): Sequential(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=32, out_features=16, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=16, out_features=8, bias=True)
    )
    (last_hidden): Identity()
    (activation): Identity()
  )
  (gmf): GMF(
    (emb_item): Embedding(3706, 32)
    (emb_user): Embedding(6040, 32)
    (hidden): Identity()
    (activation): Identity()
  )
  (neu_mf): Linear(in_features=40, out_features=1, bias=True)
  (activation): Sigmoid()
)


<br><br><br><br><br><br>

## b. Train and evaluate the recommendations accuracy of the models: 
- MF
- GMF
- MLP
- NMF

Compare the `LogLoss` and recommendations accuracy using `NDCG` and `MRR` metrics with cutoff values of 5 and 10.   
Discuss the comparison. 

**Metrics:**
- HitRatio
- nDCG
- MRR

In [ ]:
# Use your own metrics implementation OR use external packages for the metrics.
# If you are using external packages make sure they work properly. 
# A lot of the packages available does not work as you would expect.

In [ ]:
from typing import List

def compute_hit_rate(pred: np.ndarray, pred_items_ids: np.ndarray, gt_items_ids_per_user: List[int], top_n=10) -> float:
        scores = [
            compute_hit_rate_for_user(pred[i], pred_items_ids[i], gt_items_ids_per_user[i])
            for i in range(len(gt_items_ids_per_user))
        ]
        return np.mean(scores)

def compute_hit_rate_for_user(user_pred: np.ndarray,  user_pred_item_ids: np.ndarray, gt_item_id: int, top_n=10) -> int:
    top_n_indices = np.argsort(user_pred)[::-1][:top_n]
    # Get ids of the top n items
    top_n_item_ids = user_pred_item_ids[top_n_indices]
    # Check if the top n indices contains the index of the test item
    return 1 if gt_item_id in top_n_item_ids else 0
    

In [65]:
def compute_ndcg(
    pred: np.ndarray, pred_items_ids: np.ndarray, gt_items_ids_per_user: List[int], top_n=5
) -> float:
    """Computes NDCG for all users
    Args:
        pred: predicted ratings for all users
        pred_items_ids: list of item ids of the predicted ratings for all users
        gt_items_ids_per_user: list of ground truth items ids for all users
        top_n: number of top items to consider
    Returns:
        Average NDCG score for all users
    """
    scores = [
        compute_ndcg_for_user(pred[i], pred_items_ids[i], gt_items_ids_per_user[i], top_n)
        for i in range(len(gt_items_ids_per_user))
    ]
    return np.mean(scores)


def compute_ndcg_for_user(
    user_pred: np.ndarray, user_pred_item_ids: np.ndarray, gt_item_id: int, top_n=5
) -> float:
    """Computes NDCG for a single user.
    Args:
        user_pred: list of predicted ratings for a single user
        user_pred_item_ids: list of item ids of the predicted ratings for a single user
        gt_item_id id of the ground truth item
        top_n: number of top items to consider
    Returns:
        NDCG for a single user
    """
    # Get the indices of the top_n items in user pred ratings
    top_n_indices = np.argsort(user_pred)[::-1][:top_n]
    # Get the indices of the top_n items in user pred item ids
    top_n_item_ids = user_pred_item_ids[top_n_indices]

    for i in range(top_n):
        if top_n_item_ids[i] == gt_item_id:
            # We know that idcg is 1 for this case we we can just return the dcg
            return 1 / np.log2(i + 2)
    return 0


In [ ]:

def compute_mrr_for_user(user_pred: np.ndarray, user_pred_item_ids: np.ndarray, gt_item_id: int, top_n=5) -> float:
        """ Computes MRR for a single user.
        Args:
            user_pred: list of predicted ratings for a single user
            gt_item_index: index of the ground truth item
            top_n: number of top items to consider
        Returns:
            MRR for a single user
        """
        top_n_indices = user_pred[:top_n]
        # Get the indices of the top_n items in user pred item ids
        top_n_item_ids = user_pred_item_ids[top_n_indices]

        for i in range(top_n):
            if top_n_item_ids[i] == gt_item_id:
                return 1 / (i + 1)
        # if there is no match, return 0
        return 0

def compute_mrr(pred: np.ndarray, pred_items_ids: np.ndarray,  gt_items_ids_per_user: List[int], top_n=5) -> float:
        """ Mean Reciprocal Rank for all users """
        scores = [
            compute_mrr_for_user(pred[i], pred_items_ids[i], gt_items_ids_per_user[i], top_n)
            for i in range(len(gt_items_ids_per_user))
        ]
        return np.mean(scores)

**Evaluation:**

**HyperParams:**

In [51]:
# the choosen hyperparams will effect your models & your grade 
from torch.optim import lr_scheduler

#TODO: create these params for each model
# optimizer = torch.optim.Adam(model_NMF.parameters(), lr=0.001)
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)


<br><br>
Create train data:

In [ ]:
# Create pytorch DATASET for the dataset

class MovieLens(Dataset):
    def __init__(self, ratings, users, items):
        self.ratings = ratings
        self.users = users
        self.items = items

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]



In [ ]:
# Select torch device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Selected device: {device}")

<br><br>
train & eval:
- Create a training function 
- Evaluate the models trained and save the results accordingly 

In [52]:
from tqdm import tqdm

@torch.enable_grad()
def model_train(model, ratings, criterion, optimizer, scheduler, batch_size=64, num_epochs=10):
    sample_row, sample_col = ratings.nonzero()
    n_samples = len(sample_row)
    training_indices = np.arange(n_samples)
    # TODO: add other metrics arrays here for history
    train_losses_history = []

    for epoch in tqdm(range(num_epochs)):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        np.random.shuffle(training_indices)
        training_indices_batches = np.array_split(training_indices, n_samples // batch_size)
        # Convert to torch tensor
        training_indices_batches = [torch.tensor(batch) for batch in training_indices_batches]

        model.train()  # Set model to training mode

        running_loss = 0.0   # total loss of the network at each epoch
            
            # Iterate over data.
        for batch in tqdm(training_indices_batches):
            item_indices = sample_col[batch]
            user_indices = sample_row[batch]
            user_indices = user_indices.to(device)
            item_indices = item_indices.to(device)    
        

            optimizer.zero_grad()

            pred = model(user_indices, item_indices)
            loss = criterion(pred, ratings[user_indices, item_indices].float())
            loss.backward()
            optimizer.step()
                    
            running_loss += loss.item()

            if scheduler is not None:
                scheduler.step()
            
        epoch_loss = running_loss / len(training_indices_batches)
        
        print('Loss: {:.4f}'.format(epoch_loss))
            
        # save the metrics
        train_losses_history.append(epoch_loss)

    return model, train_losses_history

In [ ]:
# create eval loop
@torch.no_grad()
def model_train(model, ratings, batch_size=64):
    sample_row, sample_col = ratings.nonzero()
    n_samples = len(sample_row)
    eval_indices = np.arange(n_samples)

    model.eval()  # Set model to training mode

    np.random.shuffle(eval_indices)
    eval_indices_batches = np.array_split(eval_indices, n_samples // batch_size)

    # Convert to torch tensor
    eval_indices_batches = [torch.tensor(batch) for batch in eval_indices_batches]

    # Iterate over data.
    for batch in tqdm(eval_indices_batches):
        item_indices = sample_col[batch]
        user_indices = user_indices.to(device)
        item_indices = item_indices.to(device)
        pred = model(user_indices, item_indices)
        # TODO: calculate metrics for batch
    
    # TODO: Get overall results as an average of the metrics
    # TODO: Return metrics
    return ""

    

    


<br><br><br><br>
<br><br><br><br>
<br><br><br><br>
All Results:

In [ ]:
# df_results
# each hyperparam will add a column to the dataframe
# this is an example for a df that would allow you to create plots easily

<br><br><br><br>
**Train & Validation Loss:**

Make sure you did not overfit.  
In case you did, fix that by adding early-stopping, regularization, etc.

In [ ]:
# plot
# make sure that you did not overfit!

**Training Time:**

In [ ]:
# plot

**Metric Evaluation:**

In [ ]:
# plot suggestion/example - you may create your own plot (you should achieve higher results)

<br><br><br><br>
<br><br><br><br>

**c. How do the values of MRR and NDCG differ between your current model and the results you got in the previous exercises which implemented the explicit recommendation approach? What are the differences in preparing the dataset for evaluation?**

**d. How will you measure item similarity using the NeuMF model?**

<br><br><br>
<br><br><br>
<br><br><br>

## Question 3: Loss Function 
<br><br>

#### a. One of the enhancements presented in the Neural Collaborative Filtering paper is the usage of probabilistic activation function (the sigmoid) and binary cross entropy loss function.    

Select one of the models you implemented in question 2 and change the loss function to a `Mean Squared Error` and the activation function of the last layer to `RELU`.   

Train the model and evaluate it in a similar way to what you did in question 2. 
Compare the results and discuss.

<br><br><br><br>
<br><br><br><br>
NMFs Results:

In [ ]:
# example: df_results[df_results.model.str.startswith('NMF')]

<br><br><br>
<br><br><br>

Train & Validation Loss:

In [ ]:
# plot

<br><br><br>
Training Time:

In [ ]:
# plot

<br><br><br>
Metric Evaluation:

In [ ]:
# plot

<br><br><br><br>
**Conclusions:**

    - In
    - Your
    - Own
    - Words
    

<br><br>
<br><br>


Good Luck :)